# Research Question
For our project, we will be researching crime data in Los Angeles and how that data is affected by various variables such as COVID-19, educational attainment, and household income. Due to the global pandemic, crime rates have fluctuated substantially due to the lockdown and the reopening of the county.

# Data Sources
- Crime Data from 2020 to present, https://data.lacity.org/A-Safe-City/Crime-Data-from-2020-to-Present/2nrs-mtv8
- COVID-19 Data from 2020 to present, https://github.com/datadesk/california-coronavirus-data/blob/master/latimes-place-totals.csv
- Household Income for LA County (2018), Social Explorer
- White vs. Non-White Homeowners (2018), Social Explorer
- Mapping Inequality/ Home Owners Loan Corporation (HOLC) LA Redlining Map (1939), clsl.richmond.edu 
- LA Times - Neighborhoods, http://boundaries.latimes.com/sets/

*Note: We removed the Education Attainment dataset from Social Explorer because we decided that we have too many variables to compare.


# Data Exploration and Analysis

Now we want to explore our data sources and provide an analysis of our datasets.

## COVID-19 Rates in California
We will begin our data exploration by importing the current COVID-19 data from the LA times.

In [ ]:
import plotly.express as px
import pandas as pd

In [ ]:
latimes = pd.read_csv(
    "https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/latimes-place-totals.csv")

In [ ]:
# Now we want to get some basic statistics from the dataset. How many rows and columns?
latimes.shape

In [ ]:
#What are the first 5 rows?
latimes.head()

In [ ]:
# dataframe info?
latimes.info()

In [ ]:
# Next, we want to clean up the data. This includes empty coordinates, empty confirmed cases, and incorrect coordinates (Note: positive longitudes do not exist in California)
# We do this by using the .query() method that allows us to query and filter the dataset using SQL syntax.
latimes.query("confirmed_cases == 'NaN'")

In [ ]:
# NaN values for 'x'?
latimes.query("x == 'NaN'")

In [ ]:
#NaN values for 'y'?
latimes.query("y == 'NaN'")

In [ ]:
# Positive longitude coordinates?
latimes.query("x > 0")

In [ ]:
# Do we have any null dates?
latimes.query("date.isnull()", engine='python')

In [ ]:
# Now we will combine our arguments and clean the data:
latimes = latimes.query("confirmed_cases != 'NaN' & x < 0 & x != 'NaN' & date.notnull()", engine='python')
latimes.head()

In [ ]:
# How many records do we have now?
latimes.shape
# Less columns than before

In [ ]:
# Now we want to look at more statistics in our dataset. Let's look at confirmed cases.
latimes.confirmed_cases.describe()

In [ ]:
# Let's see which counties in California have the most confirmed cases.
latimes.groupby("county").confirmed_cases.describe().sort_values(by=["max"], ascending=False)

In [ ]:
# Since our research question is focused on Los Angeles County, let's look at which cities in LA County have the highest confirmed cases. 
latimes_LA = latimes.query("county=='Los Angeles'")

In [ ]:
latimes_LA.groupby("place").confirmed_cases.describe().sort_values(by=["max"], ascending=False).head(50)

In [ ]:
# Let's create a bar chart representing the confirmed cases in LA County overtime.
LACounty = latimes.query("county == ['Los Angeles']")
px.bar(LACounty,
      x='date',
      y='confirmed_cases')

In [ ]:
# Let's be more specific. Let's create a bar chart of the top three cities in LA County with the highest confirmed cases: Long Beach, East Los Angeles, and Pomona.
TopLA = latimes.query("place == ['Long Beach','East Los Angeles','Pomona']")
px.bar(TopLA,
      x='date',
      y='confirmed_cases',
      color = 'place')

Now that we've looked at the top three cities with the highest confirmed COVID-19 cases, let's represent our dataset in a different visualization format. 
Let's create an animated scatter plot to represent the change overtime of confirmed cases in cities across LA County.

In [ ]:
# What is the mean of  confirmed cases in LA County?
latimes_LA_mean = latimes_LA.confirmed_cases.mean()
latimes_LA_mean

In [ ]:
# Now, let's put this information on a map.
latimes_LA = latimes_LA.sort_values(by="date", ascending=True)

fig = px.scatter_geo(latimes_LA,
           lon='x',
           lat='y',
           color='confirmed_cases', 
           size='confirmed_cases',
           size_max=40, 
           hover_name='place',
           scope='usa',                     
           animation_frame='date',
           color_continuous_scale = 'RdYlGn_r',
           range_color = (0,latimes_LA_mean*2))

fig.update_geos(fitbounds="locations") 


## Crime Rates in the City of Los Angeles
Let's look at crime rates in LA County from 2020 to present. We will begin by importing the data.

In [ ]:
import pandas as pd
import plotly.express as px
from sodapy import Socrata

### Creating a Socrata Client
Next, we acquire the data using the socrata API. 
- https://dev.socrata.com/foundry/data.lacity.org/2nrs-mtv8

In [ ]:
# connect to the data portal
client = Socrata("data.lacity.org", None)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("2nrs-mtv8", limit=2000)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

# print it with .sample, which gives you random rows
df.sample(2)

In [ ]:
# Now, we want to add a "where" statement to look at the data from March 1, 2020 to April 30, 2020, limited to 30,000.
results = client.get("2nrs-mtv8", 
                     limit = 30000, 
                     where = "date_rptd between '2020-03-01T00:00:00' and '2020-04-30T00:00:00'"
                    )

In [ ]:
# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

### Data Exploration and Analysis of Crime Data

In [ ]:
# how many rows and columns?
df.shape

In [ ]:
# what fields and datatypes?
df.info()

In [ ]:
# First 5 rows?
df.head()

In [ ]:
# Let's make a bar graph with labels.
px.bar(df,
       x='date_rptd',
       title='Crime Rates in Los Angeles, March to April 2020',
       labels={'date_rptd':'Date of Crimes','count':'Number of Crimes'}
      )

In [ ]:
# first convert the date field to a datetime type
df['date_rptd'] = pd.to_datetime(df['date_rptd'])

In [ ]:
# Grouping by month and day
df_date = df.groupby([df["date_rptd"].dt.month.rename('month'),df["date_rptd"].dt.day.rename('day')],as_index=False).size().reset_index(name='count')

In [ ]:
# Let's look at the distinct value of charges
df.crm_cd_desc.unique()

In [ ]:
df_date.head()

In [ ]:
# create a label field
df_date['label'] = df_date.month.astype('str')+'-'+df_date.day.astype('str')

In [ ]:
df_date.plot(figsize=(20,10),kind="bar",x='label',y='count')

In [ ]:
# Let's clean up the labels.
px.bar(df,
       x='date_rptd',
       title='Crime Rates in Los Angeles, March to April 2020',
       labels={'date_rptd':'Date of Crimes','count':'Number of Crimes'}
      )

In [ ]:
# Let's look at  the top 25 distinct value of charges
crime_by_type = df.crm_cd_desc.value_counts().reset_index()
crime_by_type.head(25)

In [ ]:
# Rename our columns
crime_by_type.columns=['crime','count']
crime_by_type.head(25)

In [ ]:
# Let's create a horizontal bar chart.
px.bar(crime_by_type.head(25).sort_values(by='count',ascending=True),
       y='crime',
       x='count',
       orientation= 'h',
       height=800,
       width=900,
       title='Crime Rates in Los Angeles, March to April 2020')

In [ ]:
# Now, let's subset our data and begin mapping the dataset.
df.info()

Let's eliminate the unnecessary fields and create a subset of the data with just the following fields:

- `date_rptd`
- `crm_cd`
- `crm_cd_desc`
- `lat`
- `lon`

In [ ]:
# subset the data
df_mini = df[['date_rptd','crm_cd','crm_cd_desc','lat','lon']].copy()
df_mini.head()

In [ ]:
# Check the info for our subset data
df_mini.info()

In [ ]:
# Now we want to convert latitude and longitude to floats
df_mini['lat'] = df_mini['lat'].astype(float)
df_mini['lon'] = df_mini['lon'].astype(float)
df_mini.info()

In [ ]:
fig = px.scatter_mapbox(df_mini,
                        lat='lat',
                        lon='lon',
                        mapbox_style="stamen-terrain")
fig.show()

### Creating a Function
Now, we want to create a function and create multi-layered maps.

In [ ]:
import geopandas as gpd
# for basemaps
import contextily as ctx

In [ ]:
# get neighborhood boundaries from the LA Times
neighborhoods = gpd.read_file('http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/la-county-neighborhoods-v5.geojson')

In [ ]:
# trim the data to the bare minimum columns
neighborhoods = neighborhoods[['name','geometry']]
neighborhoods.head()

In [ ]:
# get the layers into a web mercator projection
# reproject to web mercator
neighborhoods = neighborhoods.to_crs(epsg=3857)

In [ ]:
# plot it!
ax=neighborhoods.plot(figsize=(12,12),
                      color='gray', 
                      edgecolor='white',
                      alpha=0.5)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

In [ ]:
# columns
list(df)

In [ ]:
# convert pandas dataframe to geodataframe
crimes = gpd.GeoDataFrame(df, 
                         crs='EPSG:4326',
                         geometry=gpd.points_from_xy(df.lon, df.lat))

In [ ]:
# get the layers into a web mercator projection
# reproject to web mercator
crimes = crimes.to_crs(epsg=3857)

In [ ]:
# drop the unmapped rows
crimes[crimes.lon==0]
crimes.drop(crimes[crimes.lon==0].index,inplace=True)

In [ ]:
# Now let's map it.
ax = crimes.plot(figsize=(12,12),color='red')
crimes.drop(crimes[crimes.lon==0].index,inplace=True)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

After multiple attempts of dropping the zero coordinate records here, we did not have success. However, we are still able to produce a two-layer map at the end.

In [ ]:
# Now we want to create a two-layered map.
# first, we define which layers will be our "base"
base = neighborhoods.plot(figsize=(12,10),
                      color='gray', 
                      edgecolor='white',
                      alpha=0.5)

# define the layer that will go on top, and add the base layer to the `ax` argument
ax = crimes.plot(ax=base, color='red', markersize=5)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

In [ ]:
# get the bounding box coordinates for the crime data
crimes.geometry.total_bounds

In [ ]:
# shortcut to put them into their own variables
minx, miny, maxx, maxy = crimes.geometry.total_bounds
print(minx)
print(maxx)
print(miny)
print(maxy)

In [ ]:
# use the bounding box coordinates to set the x and y limits
base = neighborhoods.plot(figsize=(12,12),
                          color='gray', 
                          edgecolor='white',
                          alpha=0.5)

ax = crimes.plot(ax=base, 
                color='red', 
                markersize=5
               )

ax.set_xlim(minx - 1000, maxx + 1000) # added/substracted value is to give some margin around total bounds
ax.set_ylim(miny - 1000, maxy + 1000)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

ax

In [ ]:
# subset the neighborhoods geodataframe for a single neighborhood
neighborhood = neighborhoods[neighborhoods.name=='Downtown']

# use the bounding box coordinates to set the x and y limits
minx, miny, maxx, maxy = neighborhood.geometry.total_bounds

# do a spatial join to get crime in neighborhood
crimes_in_neighborhood = gpd.sjoin(crimes,neighborhood,how='inner')

# define the base layer to be the neighborhood polygon
base = neighborhood.plot(figsize=(12,12),
                         color='red', 
                         edgecolor='red',
                         alpha=0.1)

# add the crime data, making sure to add the neighborhood polygon
ax = crimes_in_neighborhood.plot(ax=base, 
                                column='crm_cd_desc', 
                                markersize=40, 
                                legend=True,
                                cmap='tab20',
                                legend_kwds={
                                   'loc': 'upper right',
                                   'bbox_to_anchor':(1.6,1)
                                }                  # this puts the legend to the side
                            )

# set the map extent to the extent of the neighborhood bounds
ax.set_xlim(minx - 200, maxx + 200) # added/substracted value is to give some margin around total bounds
ax.set_ylim(miny - 200, maxy + 200)

# turn off the axis
ax.axis('off')

# add a title
ax.set_title('March to April 2020 Crimes in '+neighborhood.name.values[0]+' LA',fontsize=20)

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)
ax

### Spatial Autocorrelation Analysis
Next, we will use the crime data to conduct a spatial autocorrelation analysis. We will begin by importing the necessary libraries.

In [ ]:
# to read and wrangle data
import pandas as pd

# to import data from LA Data portal
from sodapy import Socrata

# to create spatial data
import geopandas as gpd

# for basemaps
import contextily as ctx

# For spatial statistics
import esda
from esda.moran import Moran, Moran_Local

import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster,plot_moran_simulation

import libpysal as lps

# Graphics
import matplotlib.pyplot as plt
import plotly.express as px

Next, we will bring in a census geography that will allow us to summarize the location of crimes committed in Los Angeles.

In [ ]:
# read downloaded geojson file from census reporter
gdf = gpd.read_file('acs2018_5yr_B01003_15000US060372711003.geojson')

In [ ]:
gdf.info()

In [ ]:
# trim the data to the bare minimum columns
gdf = gdf[['geoid','B01003001','geometry']]

# rename the columns
gdf.columns = ['FIPS','TotalPop','geometry']

In [ ]:
gdf.tail()

In [ ]:
# We want to delete last column which is for the entire city of LA
gdf=gdf.drop(2515)

In [ ]:
# fix FIPS code
gdf['FIPS'] = gdf['FIPS'].str.replace('15000US','')
gdf.tail()

In [ ]:
# sort by total pop
gdf.sort_values(by='TotalPop').head(20)

In [ ]:
# delete zero population geographies
gdf = gdf[gdf['TotalPop']>100]

Now, let's move on to mapping the census block groups.

In [ ]:
# get the layers into a web mercator projection
# reproject to web mercator
gdf = gdf.to_crs(epsg=3857)

In [ ]:
# Now, we want to plot this information
ax=gdf.plot(figsize=(12,12),
                      color='grey', 
                      edgecolor='white',
                      alpha=0.5)
# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

Now, we want to import crime data for Los Angeles from lacity.org. We will be using the timeline of March to April 2020 because the data is so dense.

In [ ]:
# Next, we will connect to the data portal
client = Socrata("data.lacity.org", None)

results = client.get("2nrs-mtv8", 
                     limit=30000,
                     where = "date_rptd between '2020-03-01T00:00:00' and '2020-04-30T00:00:00'",
                     order='crm_cd_desc')

# Convert to pandas DataFrame
crimes = pd.DataFrame.from_records(results)

In [ ]:
crimes.shape

In [ ]:
# convert pandas dataframe to geodataframe
crimes = gpd.GeoDataFrame(crimes, 
                         crs='EPSG:4326',
                         geometry=gpd.points_from_xy(crimes.lon, crimes.lat))

In [ ]:
# get the layers into a web mercator projection
# reproject to web mercator
crimes = crimes.to_crs(epsg=3857)

In [ ]:
# convert lat/lon to floats
crimes.lon = crimes.lon.astype('float')
crimes.lat = crimes.lat.astype('float')

In [ ]:
# Let's map it
ax = crimes.plot(figsize=(12,12),
                  color='red',
                  markersize=1)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

In [ ]:
# Now we want to fix the error we se above. We want to subset the zero coordinate records
crimes[crimes.lon==0]

In [ ]:
# drop the unmapped rows
crimes.drop(crimes[crimes.lon==0].index,inplace=True)

In [ ]:
# Let's map it again and see what it looks like
ax = crimes.plot(figsize=(12,12),
                  color='blue',
                  markersize=1,
                  alpha=0.5)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

Now, we want to combine both the layers we have created.

In [ ]:
# get the bounding box coordinates for the arrest data
minx, miny, maxx, maxy = crimes.geometry.total_bounds
print(minx)
print(maxx)
print(miny)
print(maxy)

In [ ]:
# set up the plot canvas with plt.subplots
fig, ax = plt.subplots(figsize=(15, 15))

# block groups
gdf.plot(ax=ax, # this puts it in the ax plot
        color='gray', 
        edgecolor='white',
        alpha=0.5)

# arrests
crimes.plot(ax=ax, # this also puts it in the same ax plot
            color='blue',
            markersize=1,
            alpha=0.2)

# use the bounding box coordinates to set the x and y limits
ax.set_xlim(minx - 1000, maxx + 1000) # added/substracted value is to give some margin around total bounds
ax.set_ylim(miny - 1000, maxy + 1000)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

Next, we want to perform a spatial join.

In [ ]:
join = gpd.sjoin(gdf, crimes, how='right')
join.head()

In [ ]:
crimes_by_gdf = join.FIPS.value_counts().rename_axis('FIPS').reset_index(name='crimes_count')

In [ ]:
crimes_by_gdf.head()

In [ ]:
# make a bar chart of top 20 geographies
crimes_by_gdf[:20].plot.bar(figsize=(20,4),
                             x='FIPS',
                             y='crimes_count')

Now, we want to join the summary table back to the GeoDataFrame.

In [ ]:
gdf=gdf.merge(crimes_by_gdf,on='FIPS')

In [ ]:
#Now, we have a count column.
gdf.head()

Now, let's look at crimes committed in Los Angeles per 1000 people.

In [ ]:
gdf['crimes_per_1000'] = gdf['crimes_count']/gdf['TotalPop']*1000

In [ ]:
gdf.sort_values(by="crimes_per_1000").tail()

In [ ]:
# map the top 20 geographies
ax = gdf.sort_values(by='crimes_per_1000',ascending=False)[:20].plot(figsize=(12,10),
                                                             color='blue',
                                                             edgecolor='white',
                                                             alpha=0.5,legend=True)


# title
ax.set_title('Top 20 locations of Crimes Committed in Los Angeles per 1000 people (March-April 2020)')

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

In [ ]:
#Let's make a choropleth map
ax = gdf.plot(figsize=(15,15),
                        column='crimes_per_1000',
                        legend=True,
                        alpha=0.8,
                        cmap='Blues',
                        scheme='quantiles')

ax.axis('off')
ax.set_title('March to April 2020 crimes committed in LA per 1000 people',fontsize=22)
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

Now, we want to conduct spatial autocrrelation to determine what degree an existing pattern could potentially be random.

We will be using Moran's I statistic.

In [ ]:
wq =  lps.weights.KNN.from_dataframe(gdf,k=8)
wq.transform = 'r'

In [ ]:
# create a new column for the spatial lag
gdf['crimes_per_1000_lag'] = lps.weights.lag_spatial(wq, gdf['crimes_per_1000'])

In [ ]:
gdf.sort_values(by='crimes_per_1000',ascending=False).sample(10)

Now, let's consider the donut and diamond geographies to better understand the significance of the spatial lag values.

In [ ]:
gdf[gdf['FIPS'].isin(['060372739022', '060379800241'])]

In [ ]:
# set the mapbox access token
token = 'pk.eyJ1IjoiZGhleWRhciIsImEiOiJja2llM3k3bHExYTN5Mnlueng0ZTd6bGQ2In0.TIg8HNU19SXwuGSdJSkRtQ'
px.set_mapbox_access_token(token)

In [ ]:
# subset donut, project to WGS84, and get its centroid
gdf_donut = gdf[gdf.FIPS=='060372739022']
gdf_donut = gdf_donut.to_crs('epsg:4326')

# what's the centroid?
minx, miny, maxx, maxy = gdf_donut.geometry.total_bounds
center_lat_donut = (maxy-miny)/2+miny
center_lon_donut = (maxx-minx)/2+minx

In [ ]:
# subset diamond, project to WGS84, and get its centroid
gdf_diamond = gdf[gdf.FIPS=='060379800241']
gdf_diamond = gdf_diamond.to_crs('epsg:4326')

# what's the centroid?
minx, miny, maxx, maxy = gdf_diamond.geometry.total_bounds
center_lat_diamond = (maxy-miny)/2+miny
center_lon_diamond = (maxx-minx)/2+minx

In [ ]:
px.choropleth_mapbox(gdf_donut, 
                     geojson=gdf_donut.geometry, 
                     locations=gdf_donut.index, 
                     mapbox_style="satellite-streets",
                     zoom=14, 
                     center = {"lat": center_lat_donut, "lon": center_lon_donut},
                     hover_data=['crimes_count','crimes_per_1000','crimes_per_1000_lag'],
                     opacity=0.4,
                     title='The Donut')

In [ ]:
px.choropleth_mapbox(gdf_diamond, 
                     geojson=gdf_diamond.geometry, 
                     locations=gdf_diamond.index, 
                     mapbox_style="satellite-streets",
                     zoom=12, 
                     center = {"lat": center_lat_diamond, "lon": center_lon_diamond},
                     hover_data=['crimes_count','crimes_per_1000','crimes_per_1000_lag'],
                     opacity=0.4,
                     title='The Diamond')

Now, we want to map the entire dataframe with the new spatial lag column we created.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))

# spatial lag choropleth
gdf.plot(ax=ax,
         figsize=(15,15),
         column='crimes_per_1000_lag',
         legend=True,
         alpha=0.8,
         cmap='Blues',
         scheme='quantiles')

# uncomment this to see the actual point locations of arrests
# arrests.plot(ax=ax, 
#              color='blue',
#              markersize =1,
#              alpha=0.2, 
#              legend=True)

ax.axis('off')
ax.set_title('March-April 2020 Crimes Committed in Los Angeles per 1000 people',fontsize=22)

ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

Let's compare this with the spatial lag map.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 8))

ax1, ax2 = axs

# regular count map on the left
gdf.plot(column='crimes_per_1000', 
            cmap='Blues', 
            scheme='quantiles',
            k=5, 
            edgecolor='white', 
            linewidth=0., 
            alpha=0.75, 
            ax=ax1 
           )


ax1.axis("off")
ax1.set_title("Crimes Committed in Los Angeles per 1000")

# spatial lag map on the right
gdf.plot(column='crimes_per_1000_lag', 
            cmap='Blues', 
            scheme='quantiles',
            k=5, 
            edgecolor='white', 
            linewidth=0., 
            alpha=0.75, 
            ax=ax2 
           )

ax2.axis("off")
ax2.set_title("Crimes Committed in Los Angeles per 1000 - Spatial Lag")

plt.show()

Next, we want to create an interactive spatial lag satellite map.

In [ ]:
# interactive version needs to be in WGS84
gdf_web = gdf.to_crs('EPSG:4326')

In [ ]:
# what's the centroid?
minx, miny, maxx, maxy = gdf_web.geometry.total_bounds
center_lat_gdf_web = (maxy-miny)/2+miny
center_lon_gdf_web = (maxx-minx)/2+minx

In [ ]:
# the median?
median = gdf_web.crimes_per_1000_lag.median()

In [ ]:
fig = px.choropleth_mapbox(gdf_web, 
                     geojson=gdf_web.geometry, 
                     locations=gdf_web.index, 
                     mapbox_style="satellite-streets",
                     zoom=9, 
                     color='crimes_per_1000_lag',
                     color_continuous_scale='Blues',
                     color_continuous_midpoint =median,
                     range_color =(0,median*2),
                     hover_data=['crimes_count','crimes_per_1000','crimes_per_1000_lag'],
                     center = {"lat": center_lat_gdf_web, "lon": center_lon_gdf_web},
                     opacity=0.8,
                     width=1200,
                     height=800,
                     labels={
                             'crimes_per_1000_lag':'Crimes Committed in Los Angeles per 1000 (Spatial Lag)',
                             'crimes_per_1000':'Crimes Committed in Los Angeles per 1000',
                     })
fig.update_traces(marker_line_width=0.1, marker_line_color='white')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

Let's move towards the Moran's Plot.

In [ ]:
y = gdf.crimes_per_1000
moran = Moran(y, wq)
moran.I

We have a positive spatial autocorrelation, which means high values are close to high values, or low values are close to low values.

In [ ]:
fig, ax = moran_scatterplot(moran, aspect_equal=True)
plt.show()

In [ ]:
plot_moran_simulation(moran,aspect_equal=False)

In [ ]:
moran.p_sim

We can reject the hypothesis that the map is random becuase the p-value is small.

Now, we want Local Indicators of Spatial Association (LISA) to calculate the clusters by classifying it into these 4 groups: High values near to high values (HH), low values with nearby low values (LL), low values with high values (LH) and high values with nearbly low values (HL)

In [ ]:
# calculate local moran values
lisa = esda.moran.Moran_Local(y, wq)

In [ ]:
# Plot
fig, ax = moran_scatterplot(lisa, p=0.05)
ax.set_xlabel("Crimes Committed")
ax.set_ylabel('Spatial Lag of Crimes Committed')
plt.text(1.95, 0.5, "HH", fontsize=25)
plt.text(1.95, -1.5, "HL", fontsize=25)
plt.text(-2, 1, "LH", fontsize=25)
plt.text(-1, -1, "LL", fontsize=25)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14,12))
lisa_cluster(lisa, gdf, p=0.05, ax=ax)
plt.show()

Let's compare the different p-values.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 8))

ax1, ax2 = axs

# regular count map on the left
lisa_cluster(lisa, gdf, p=0.05, ax=ax1)

ax1.axis("off")
ax1.set_title("P-value: 0.05")

# spatial lag map on the right
lisa_cluster(lisa, gdf, p=0.01, ax=ax2)
ax2.axis("off")
ax2.set_title("P-value: 0.01")

plt.show()

Now, let's create an interactive version of the LISA map.

In [ ]:
# original value list
lisa.y[:5]

In [ ]:
# quadrant list
lisa.q[:5]

In [ ]:
# p sim list
lisa.p_sim[:5]

In [ ]:
# add quadrant numbers to the dataframe
gdf['q'] = lisa.q.tolist()

In [ ]:
# add individual p-values to the dataframe
gdf['p_sim'] = lisa.p_sim.tolist()

In [ ]:
gdf.head()

In [ ]:
# Let's create a hotspot map
# identify just the hotspot geographies
hot_spots = gdf[(gdf.p_sim < 0.05) & (gdf.q == 1)]

In [ ]:
hot_spots.shape

In [ ]:
hot_spots.plot(figsize=(12,12),color='blue',legend=True,categorical=True)

In [ ]:
# interactive version needs to be in WGS84
hot_spots = hot_spots.to_crs('EPSG:4326')

In [ ]:
# what's the centroid?
minx, miny, maxx, maxy = hot_spots.geometry.total_bounds
center_lat_hot_spots = (maxy-miny)/2+miny
center_lon_hot_spots = (maxx-minx)/2+minx

In [ ]:
fig = px.choropleth_mapbox(hot_spots, 
                     geojson=hot_spots.geometry, 
                     locations=hot_spots.index, 
                     mapbox_style="satellite-streets",
                     center = {"lat": center_lat_hot_spots, "lon": center_lon_hot_spots},
                     zoom=9, 
                     opacity=0.6,
                     color='crimes_per_1000_lag',
                     color_continuous_scale='Blues',
                     color_continuous_midpoint =median,
                     range_color =(0,median*2),
                     hover_data=['crimes_count','crimes_per_1000','crimes_per_1000_lag'],
                    width=1200,
                     height=800,
                           labels={
                             'crimes_per_1000_lag':'Crimes Committed in Los Angeles per 1000 (Spatial Lag)',
                             'crimes_per_1000':'Crimes Committed in Los Angeles per 1000',
                     })
fig.update_traces(marker_line_width=1, marker_line_color='white')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# Group Contributions
1. Donna Heydar (Donna contributed to breaking down the educational attainment data in Los Angeles County, however we decided not to include the educational attainment dataset in our final project. Donna also contributed to the data exploration and analysis of Crime data in LA as well as COVID-19 data.) 
2. Daniel Ruiz ( Daniel also contributed to the data exploration and analysis of HOLC Redlining. He also contributed to the comparison between income and homeownership in Los Angeles.) 